In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pysat

In [3]:
!pip3 install pysat
!pip3 install cnf

In [4]:
!pip install python-sat

In [7]:
!pip install pycosat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycosat: filename=pycosat-0.6.6-cp310-cp310-linux_x86_64.whl size=169339 sha256=79758113cc825b77e1a992927a6f088395fab639e520c253609d97a4324cb904
  Stored in directory: /root/.cache/pip/wheels/63/29/df/b8c22ca5812e2d7b342269a53add280b5bad42a540f34c3dc1
Successfully built pycosat


In [8]:
from itertools import product
from pysat.formula import CNF
from pysat.solvers import Glucose3
from pycosat import solve

def get_board_size():
    # Prompt user to enter the size of the board (N)
    while True:
        try:
            n = int(input("Enter the size of the board (N): "))
            if n >= 4:
                return n
            print("N must be at least 4.")
        except ValueError:
            print("Please enter a valid positive integer.")

def create_variables(n):
    # Create a mapping of board positions to unique variable numbers
    vars = {}
    counter = 1
    for row, col in product(range(n), range(n)):
        vars[(row, col)] = counter
        counter += 1
    return vars

def generate_constraints(n, vars):
    # Generate constraints for the N-Queens problem
    constraints = CNF()

    # One queen per row
    for row in range(n):
        row_constraints = [vars[(row, col)] for col in range(n)]
        constraints.append(row_constraints)
        for col1 in range(n):
            for col2 in range(col1 + 1, n):
                constraints.append([-vars[(row, col1)], -vars[(row, col2)]])

    # One queen per column
    for col in range(n):
        col_constraints = [vars[(row, col)] for row in range(n)]
        constraints.append(col_constraints)
        for row1 in range(n):
            for row2 in range(row1 + 1, n):
                constraints.append([-vars[(row1, col)], -vars[(row2, col)]])

    # No two queens on the same diagonal
    for row in range(n):
        for col in range(n):
            for step in range(1, n):
                if row + step < n and col + step < n:
                    constraints.append([-vars[(row, col)], -vars[(row + step, col + step)]])
                if row + step < n and col - step >= 0:
                    constraints.append([-vars[(row, col)], -vars[(row + step, col - step)]])
                if row - step >= 0 and col + step < n:
                    constraints.append([-vars[(row, col)], -vars[(row - step, col + step)]])
                if row - step >= 0 and col - step >= 0:
                    constraints.append([-vars[(row, col)], -vars[(row - step, col - step)]])

    return constraints

def solve_nqueens(constraints):
    # Use the Glucose3 solver to solve the N-Queens problem with the given constraints
    with Glucose3(bootstrap_with=constraints) as solver:
        if solver.solve():
            return solver.get_model()  # Return the solution if found
    return None  # Return None if no solution exists

def display_solution(n, solution, vars):
    # Display the solution on the board
    if solution:
        # Initialize an empty board
        board = [['.' for _ in range(n)] for _ in range(n)]
        for var in solution:
            if var > 0:
                row, col = next((key for key, value in vars.items() if value == var), None)
                board[row][col] = 'Q'  # Place a queen at the corresponding position
        for row in board:
            print(' '.join(row))  # Print the board row by row
    else:
        print("No solution exists")

if __name__ == "__main__":
    # Main function to execute the N-Queens solver
    n = get_board_size()  # Get the size of the board from the user
    vars = create_variables(n)  # Create variables for the board positions
    constraints = generate_constraints(n, vars)  # Generate constraints for the N-Queens problem
    solution = solve_nqueens(constraints)  # Solve the problem using the constraints
    display_solution(n, solution, vars)  # Display the solution


Enter the size of the board (N): 6 7
Please enter a valid positive integer.
Enter the size of the board (N): 8
. . . Q . . . .
. Q . . . . . .
. . . . . . Q .
. . Q . . . . .
. . . . . Q . .
. . . . . . . Q
. . . . Q . . .
Q . . . . . . .
